<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hand-made-agent" data-toc-modified-id="Hand-made-agent-1">Hand made agent</a></span><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1.1">Goal</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-1.2">Imports</a></span></li><li><span><a href="#Simple-agent" data-toc-modified-id="Simple-agent-1.3">Simple agent</a></span></li><li><span><a href="#Research-agent" data-toc-modified-id="Research-agent-1.4">Research agent</a></span></li><li><span><a href="#Build-new-workers" data-toc-modified-id="Build-new-workers-1.5">Build new workers</a></span></li><li><span><a href="#Build-new-city-tiles" data-toc-modified-id="Build-new-city-tiles-1.6">Build new city tiles</a></span><ul class="toc-item"><li><span><a href="#Deterministic-choice-of-closest-tile" data-toc-modified-id="Deterministic-choice-of-closest-tile-1.6.1">Deterministic choice of closest tile</a></span></li><li><span><a href="#Random-choice-of-closest-tile" data-toc-modified-id="Random-choice-of-closest-tile-1.6.2">Random choice of closest tile</a></span></li></ul></li><li><span><a href="#Avoid-collisions-when-moving" data-toc-modified-id="Avoid-collisions-when-moving-1.7">Avoid collisions when moving</a></span><ul class="toc-item"><li><span><a href="#Study-blocking" data-toc-modified-id="Study-blocking-1.7.1">Study blocking</a></span></li><li><span><a href="#Visualize-solution" data-toc-modified-id="Visualize-solution-1.7.2">Visualize solution</a></span></li></ul></li><li><span><a href="#How-I-would-play-as-a-human?" data-toc-modified-id="How-I-would-play-as-a-human?-1.8">How I would play as a human?</a></span></li><li><span><a href="#Challenges-found-when-creating-an-agent" data-toc-modified-id="Challenges-found-when-creating-an-agent-1.9">Challenges found when creating an agent</a></span></li><li><span><a href="#Learnings" data-toc-modified-id="Learnings-1.10">Learnings</a></span></li><li><span><a href="#Open-Questions" data-toc-modified-id="Open-Questions-1.11">Open Questions</a></span></li><li><span><a href="#Summary" data-toc-modified-id="Summary-1.12">Summary</a></span></li><li><span><a href="#TODO" data-toc-modified-id="TODO-1.13">TODO</a></span></li></ul></li></ul></div>

# Hand made agent

## Goal

## Imports

In [1]:
# Use this to reload changes in python scripts
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from kaggle_environments import make
from functools import partial

from luxai.utils import render_game_in_html
from luxai.agents.basic import (
    SimpleAgent,
    ResearchAgent,
    BuildWorkerOrResearchAgent,
    NaiveViralAgent,
    NaiveRandomViralAgent,
    ViralRemoveBlockingAgent,
)

Loading environment football failed: No module named 'gfootball'


In [3]:
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game import Game
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_map import Cell, RESOURCE_TYPES
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.constants import Constants
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux.game_constants import GAME_CONSTANTS
from kaggle_environments.envs.lux_ai_2021.test_agents.python.lux import annotate

In [4]:
plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (20, 5)  
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Simple agent

I will first refactorize the simple agent to ease up the improvement process.

The visualization below shows that it acts exactly as the simple_agent, but the code is much better prepared for expansion.

In [5]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 0, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run(['simple_agent', SimpleAgent()])
render_game_in_html(env)

Opening in existing browser session.


## Research agent

Let's start by improving the simple agent to do research whith the available city tiles. I will first refactorize the simple agent to ease up the improvement process.

In [5]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 0, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run(['simple_agent', ResearchAgent()])
render_game_in_html(env)

Opening in existing browser session.


Metrics show that the new agent is researching while the other isn't.

In [6]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 1, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run(['simple_agent', ResearchAgent()])
render_game_in_html(env)

Opening in existing browser session.


[13002:13002:0901/153446.195032:ERROR:broker_posix.cc(43)] Invalid node channel message
[13000:13000:0100/000000.376063:ERROR:broker_posix.cc(43)] Invalid node channel message


With this seed we can see that the worker dies in the night because it has no fuel, let's create a new agent that is able to generate workers when possible, otherwise researchs.

## Build new workers

If I want to build new workers I have to first check if there are more city tiles than units.

In [6]:
env = make("lux_ai_2021", debug=True, configuration={'width': 12, 'height': 12, 'seed': 1, 'episodeSteps': 361})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), ResearchAgent()])
render_game_in_html(env)

Opening in existing browser session.


This seed of one allows to see that the first agent is able to regenerate the agent when it dies, and the other agent cannot.

## Build new city tiles

Building new city tiles is more complex. As a first step I'm only going to grow an existing city, not to build a new one.

Difficulties that I find:

- Where to build a new city tile? I could simply look for empty adjacent locations to the city and choose the one that is more optimal
- How to decide when to build a new city tile? I could build a new city tile if I have enough fuel to survive the night after creating the new tile. But a new tile will create a new worker that could bring more resources. I need to find a compromise between carrying resources to the city or growing it.
- When going to build a new city tile I have to avoid entering to the city or I will lost the resources

Better explained:

- When: balance between gathering resources and building
- Where: deciding the place for a new location is not trivial. Next to an existing city? The closest empty place to the worker?
- Who: Which worker will do the task?
- How: How to go to the location of the new city? City tiles should be avoided because resources will be converted to fuel

As a first step I'm going to design a viral agent:

1. Gather resources until the cargo is full
2. Go to the closest empty location and build a city tile

I could randomly choose between building a new city tile or carrying resources using a parameter.

### Deterministic choice of closest tile

In [5]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), NaiveViralAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


It's funny because although it is able to collect more resources and do more research it loses because of collisions. All the agents want to move to the same location, get blocked and die.

In [6]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 0, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), NaiveViralAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


I need a way to avoid collisions and maybe randomizing the tiles when searching for the closest one could be better.

### Random choice of closest tile

In [8]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), NaiveRandomViralAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


In [7]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 0, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), NaiveRandomViralAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


This agent is much more capable than the deterministic one because it gets blocked less frequently. However, we have to solve the blocking problem.

## Avoid collisions when moving

The previous agent has fail because agents want to move to the same location and they get blocked.

One simple way is to remove actions that lead to the same place (although clearly not optimal). Let's start by doing that.

### Study blocking

In [9]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), NaiveViralAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


In [10]:
game_info[55][1]

{'action': ['m u_2 s', 'bcity u_3', 'm u_4 s'],
 'reward': 40003,
 'info': {},
 'observation': {'remainingOverageTime': 60, 'reward': 40003, 'player': 1},
 'status': 'ACTIVE'}

If we look at the actions we don't have an easy way to find if they are moving to the same location. Thus I believe it is better to save the position of the movement and check that later previosly to generating the actions. Important facts:

- There could be multiple units on a city, so those movements should not be considered for this checking
- A unit that does not move could also be blocking the path

### Visualize solution

In [5]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([BuildWorkerOrResearchAgent(), ViralRemoveBlockingAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


In [6]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 1, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([NaiveRandomViralAgent(1), ViralRemoveBlockingAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


In [7]:
env = make("lux_ai_2021", debug=True, 
           configuration={'width': 12, 'height': 12, 'seed': 0, 
                          'episodeSteps': 361, 'annotations':True})
initial_info = env.reset()
game_info = env.run([NaiveRandomViralAgent(1), ViralRemoveBlockingAgent(1)])
render_game_in_html(env)

Opening in existing browser session.


## How I would play as a human?

## Challenges found when creating an agent

Let's enumerate the difficulties we have found when trying to create an agent by hand.

- Agent dying in the night because it does not have resources to create light (seed 1)
- How to avoid collisions between units, how to coordinate the different goals of the units
- How to move to a new location avoiding obstacles

## Learnings

- I have realized that building new city tiles is much cheaper than surviving the night (100 resources vs 300 resources). That explains why agents in the forum do "only" worry about increasing the worker population and not about preserving houses.
- However I believe that maybe once coal or uranium are available it may have sense to preserve cities since using wood for cities is cheaper. Maybe carts could play a role there.

## Open Questions

- To build a new city tile I need 100 resources. If I understand correctly that makes "cheaper" to build cities with wood rather than with coal or uranium.
- Does moving to center position increases the cooldown of the agent?

## Summary

## TODO

- Move code to the library
- Add tests 
- Function that annotates the resources of the map